<a href="https://colab.research.google.com/github/Bhanudutta/WrapLap/blob/master/CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Running this may take sometime, this is for initially setting up CUDA for COLAB
#takes around 6 minutes
#Uninstall any previous versions of CUDA completely.
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
#Install CUDA Version 9.
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
#check nvcc version
!nvcc --version
#cuda magic tool
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-cg-toolkit' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-390' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-410' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-415' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-418' for glob 'nvidia*'
Note, selecting 'nvidi

In [0]:
#test run
#searching
%%cu
#include<iostream>
#include<ctime>
void random_ints(int*, int);

__global__ void find(int *a,int k,int *first)
{
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    if(a[index] == k)
        {
            if(index<*first)
            {
                *first = index;
            }
        }
}
#define N (2048*2048)
#define THREADS_PER_BLOCK 4096
int main(void) 
{
    int *a,f=N;
    int *d_a,*d_f;
    int size = N * sizeof(int);
 
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_f,sizeof(int));
 
    a = (int *)malloc(size); random_ints(a, N);
 
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_f, &f, sizeof(int), cudaMemcpyHostToDevice);
 
    int k=4100000;
    const clock_t begin_time = clock();
    find<<<N/THREADS_PER_BLOCK,THREADS_PER_BLOCK>>>(d_a,k,d_f);
    cudaMemcpy(&f, d_f, sizeof(int), cudaMemcpyDeviceToHost);
    std::cout <<"Time "<< float( clock () - begin_time )/CLOCKS_PER_SEC<<"\n";
    std::cout<<"First occurance is "<<f;
    
    cudaFree(d_a); 
    cudaFree(d_f);
    free(a);
 
    return 0;
}

void random_ints(int* a, int n)
{
   int i;
   for (i = 0; i < n; ++i)
    a[i] = i;//rand();
}

Time 9.1e-05
First occurance is 4194304


In [0]:
%%cu
#include<iostream>
#include<ctime>
#define N (2048*2048)

void random_ints(int*, int);

int main()
{
    int *a;
    int size = N * sizeof(int);
    a = (int *)malloc(size); random_ints(a, N);
    int k=4100000;
    const clock_t begin_time = clock();
    for(unsigned i=0;i<N;i++)
    {
        if(a[i] == k)
            {
                std::cout<<"At :"<<i;
                break;
            }
    }
    std::cout <<"Time "<< float( clock () - begin_time )/CLOCKS_PER_SEC<<"\n";
    free(a);
}
void random_ints(int* a, int n)
{
   int i;
   for (i = 0; i < n; ++i)
    a[i] = i;//rand();
}

At :4100000Time 0.010743



In [0]:
#sorter
%%cu
#include<iostream>


namespace CUDA
{
    __global__ void add(int *a,int *b,int *c)
    {
        *c = *a+*b;
    }

    __global__ void v_add(int *a,int *b,int *c)
    {
        int index = threadIdx.x + blockIdx.x * blockDim.x;
        c[index] = a[index]+b[index];
    }

    template<typename T>
    class Var
    {
        protected:
        T *t;
        bool autofree;
        private:
        void autofreeon()
        {
            autofree = true;
        }
        void autofreeoff()
        {
            autofree = false;
        }
        void Init()
        {
            cudaMalloc((void **)&t, sizeof(T));
            autofreeon();
        }
        void assign(T x)
        {
            cudaMemcpy(t, &x, sizeof(T), cudaMemcpyHostToDevice);
        }
        T get()
        {
            T x;
            cudaMemcpy(&x,t, sizeof(T), cudaMemcpyDeviceToHost);
            return x;
        }
        public:
        Var()
        {
            Init();
        }
        Var(T x)
        {
            Init();
            assign(x);
        }
        Var(const Var &v)
        {
            t = v.t;
            autofree = v.autofree;
        }
        void operator=(T x)
        {
            assign(x);
        }
        operator T()
        {
            return get();
        }
        void free()
        {
            cudaFree(t);
        }
        ~Var()
        {
            if(autofree)
                free();
            else autofreeon();
        }
        Var operator+(Var v)
        {
            Var x;
            x.autofreeoff();
            add<<<1,1>>>(this->t,v.t,x.t);
            return x;
        }
    };
    
    template<typename T>
    class Vector
    {
        protected:
        T *t;
        int size;
        bool autofree;
        private:
        void autofreeon()
        {
            autofree = true;
        }
        void autofreeoff()
        {
            autofree = false;
        }
        void Init(int s)
        {
            size = s;
            cudaMalloc((void **)&t, sizeof(T)*size);
            autofreeon();
        }
        void assign(T *x)
        {
            cudaMemcpy(t, x, sizeof(T)*size, cudaMemcpyHostToDevice);
        }
        T get()
        {
            T *x;
            x = new T[size];
            cudaMemcpy(x,t, sizeof(T)*size, cudaMemcpyDeviceToHost);
            return x;
        }
        public:
        Vector(int s)
        {
            Init(s);
        }
        Vector(T *x,int s)
        {
            Init(s);
            assign(x);
        }
        Vector(const Vector &v)
        {
            t = v.t;
            size = v.size;
            autofree = v.autofree;
        }
        void operator=(T *x)
        {
            assign(x);
        }

        void free()
        {
            cudaFree(t);
        }
        ~Vector()
        {
            if(autofree)
                free();
            else autofreeon();
        }
        Vector operator+(Vector v)
        {
            Vector x;
            x.autofreeoff();
            add<<<1,size>>>(this->t,v.t,x.t);
            return x;
        }
    };
};

int main()
{
    /*int y = 10;
    CUDA::Var<int> a;
    a=10;
    printf("%d\n",(int)a);
    CUDA::Var<int> b;
    b=20;
    printf("%d\n",(int)b);
    CUDA::Var<int> c;
    c=a+b;
    printf("%d\n",(int)c);
    */
    
}

/tmp/tmpp0ab1g3h/8cb8ab7a-acea-4bb2-a47e-3fca43910dae.cu(110): error: argument list for class template "CUDA::Var" is missing

/tmp/tmpp0ab1g3h/8cb8ab7a-acea-4bb2-a47e-3fca43910dae.cu(110): error: expected an identifier

/tmp/tmpp0ab1g3h/8cb8ab7a-acea-4bb2-a47e-3fca43910dae.cu(111): error: expected a ";"

/tmp/tmpp0ab1g3h/8cb8ab7a-acea-4bb2-a47e-3fca43910dae.cu(127): warning: parsing restarts here after previous syntax error

/tmp/tmpp0ab1g3h/8cb8ab7a-acea-4bb2-a47e-3fca43910dae.cu(128): error: expected a ";"

/tmp/tmpp0ab1g3h/8cb8ab7a-acea-4bb2-a47e-3fca43910dae.cu(132): warning: variable "y" was declared but never referenced

4 errors detected in the compilation of "/tmp/tmpxft_00003b7a_00000000-8_8cb8ab7a-acea-4bb2-a47e-3fca43910dae.cpp1.ii".

